In [20]:
file_path = "CDS_aidata"

with open(file_path, "r", encoding="utf-8") as f:
    preview_lines = [next(f) for _ in range(50)]

preview_lines


['1-----\n',
 'SENTENCE: what:WH can:AUX she:PRON play:V your:DET shoes:N \n',
 'SEM_REP:  ,,question#0,auxiliary#0,pronoun#0,action#0,compete#0,determiner#0,clothing#0,shoes#0\n',
 '1-----\n',
 'SENTENCE: look:V the:DET loud:ADJ shoes:N \n',
 'SEM_REP:  ,,perceive#0,determiner#0,property#0,sound#0,clothing#0,shoes#0\n',
 '1-----\n',
 'SENTENCE: it:PRON open:V gently:ADV \n',
 'SEM_REP:  ,,pronoun#0,change#0,manner#0\n',
 '1-----\n',
 'SENTENCE: where:WH can:AUX you:PRON wash:V the:DET cookie:N \n',
 'SEM_REP:  ,,question#0,auxiliary#0,pronoun#0,cleaning#0,determiner#0,food#0,cookie#0\n',
 '1-----\n',
 'SENTENCE: what:WH be:AUX they:PRON eat:V \n',
 'SEM_REP:  ,,question#0,auxiliary#0,pronoun#0,consume#0\n',
 '1-----\n',
 'SENTENCE: close:V here:ADV \n',
 'SEM_REP:  ,,change#0,location#0\n',
 '1-----\n',
 'SENTENCE: close:V your:DET clean:ADJ block:N \n',
 'SEM_REP:  ,,change#0,determiner#0,property#0,cleanliness#0,object#0,block#0\n',
 '1-----\n',
 'SENTENCE: wow:INTJ \n',
 'SEM_REP: 

In [1]:
import sys
import os
import re
import getopt

In [2]:
import evaluate
import wmmapping
import numpy
import pickle

 Using wmmapping_fixed.py


In [3]:
import learnconfig_fixed as learnconfig

In [4]:
import learn_fixed as learn

In [5]:
import sys

sys.argv = [
    'main.py',
    '-c', 'CDS_aidata',
    '-l', 'all_catf_norm_prob_lexicon_cs.all',
    '-i', 'input_dir',
    '-o', 'output_dir',
    '-C', 'config.ini',
    '-s', 'stopwords.txt',
]

def main():
    try:
        options_list = ["help", "corpus=", "lexicon=", "inputdir=", "output=", "config=", "stop=", "f="]
        opts, args = getopt.getopt(sys.argv[1:], "hc:l:i:o:C:s:f:", options_list)
    except getopt.GetoptError as msg:
        print(msg)
        usage()
        sys.exit(2)

    if len(opts) < 4:
        usage()
        sys.exit(0)

    corpus_path = ""
    stop = ""
    config_path = ""
    lexname = ""
    indir = ""
    outdir = ""

    for o, a in opts:
        if o in ("-h", "--help"):
            usage()
            sys.exit(0)
        elif o in ("-c", "--corpus"):
            corpus_path = a
        elif o in ("-l", "--lexicon"):
            lexname = a
        elif o in ("-i", "--inputdir"):
            indir = a
        elif o in ("-o", "--output"):
            outdir = a
        elif o in ("-C", "--config"):
            config_path = a
        elif o in ("-s", "--stop"):
            stop = a
        elif o in ("-f", "--f"):
            pass

    config_path = "config.ini"
    stop = "stopwords.txt"
    lexname = "all_catf_norm_prob_lexicon_cs.all"
    corpus_path = "CDS_aidata"


    learner_config = learnconfig.LearnerConfig(config_path)
    print("ww")
    stopwords = []
    if len(stop) > 2:
        with open(stop, 'r') as stopwords_file:
            for line in stopwords_file:
                stopwords.append(line.strip() + ":N")

    learner = learn.Learner(lexname, learner_config, stopwords)

    if not os.path.exists(outdir):
        os.makedirs(outdir)

    if corpus_path == "":
        for dirpath, _, filenames in os.walk(indir):
            for fname in filenames:
                if re.search(r"#\d+\.txt$", fname):
                    outdirpath = dirpath.replace(indir, outdir)
                    if not os.path.exists(outdirpath):
                        os.makedirs(outdirpath)
                    corpus = os.path.join(dirpath, fname)
                    print("Processing Corpus:", corpus)
                    learner.process_corpus(corpus, outdirpath)
                    print("Done")
                    learner.reset()
    else:
        learner.process_corpus(corpus_path, outdir)

    if learner._stats_flag:
        print("output files ..... ")
        write_acq_score_timestamp(learner, outdir)
        print("alignment", learner.alignment_method, corpus_path)

    write_learned_lex(learner, outdir)
    write_alignments(learner, outdir)


In [6]:
def write_alignments(learner, outdir):
    """
    Write the contents of the learner to a file in directory outdir.
    The file is named as: aligns_lm_{lambda}_a{alpha}_ep{epsilon} where lambda,
    alpha, and epsilon come from the Learner learner.

    The file is written as:
        {word}--{feature} [ ({time}, {alignment}), ({time}, {alignment}), ... ]
    for each word-feature pair that occurred at least once together.
    """
    lm = learner._lambda
    a = learner._alpha
    ep = learner._epsilon
    filename = f"{outdir}/aligns_lm_{lm}_a{a}_ep{ep}"

    with open(filename, 'w') as output:
        for word in learner._wordsp.all_words(0):
            for feature in learner._features:
                alignments = learner._aligns.alignments(word, feature)
                if not alignments:
                    continue

                line = f"{word}--{feature} [ "
                for t, val in alignments.items():
                    line += f"({t}, {val:.5f}), "
                line += " ]\n\n"
                output.write(line)


In [7]:
def write_learned_lex(learner, outdir):
    """
    Write the contents of the learner to a file in directory outdir.
    The file is named as: lex_lm_{lambda}_a{alpha}_ep{epsilon} where lambda,
    alpha, and epsilon come from the Learner learner.

    The file is written as:
        {word}:{frequency} [{feature}:(true_prob, learned_prob), ...]
        <<acquisition_score>>
    for each word.
    """
    lm = learner._lambda
    a = learner._alpha
    ep = learner._epsilon
    filename = f"{outdir}/lex_lm_{lm}_a{a}_ep{ep}"
    min_prob = 0.0001

    with open(filename, 'w') as output:
        for word in learner._wordsp.all_words(0):
            prob_feature_pairs = learner._gold_lexicon.meaning(word).sorted_features()
            learned_meaning = learner._learned_lexicon.meaning(word)
            freq = learner._wordsp.frequency(word)

            line = f"{word}:{freq} ["
            for true_prob, feature in prob_feature_pairs:
                if true_prob > min_prob:
                    learned_prob = learned_meaning.prob(feature)
                    line += f"{feature}:(%.5f, %.5f), " % (true_prob, learned_prob)
            line += " ]\n\n"
            output.write(line)

            acq_score = learner.acquisition_score(word)
            output.write(f"   << {acq_score:.6f} >>\n\n")


In [8]:
def write_acq_score_timestamp(learner, outdir):
    '''
    added by Shanshan Huang
    Record the acquisition score in each time step in pickle format,
    with the option to save the whole learner object into a single pickle
    (warning: very large!!!!)
    These pickle files are later used to plot learning graphs used for
    analysis in the paper
    '''


    with open(outdir+'/acq_score_timestamp.pkl', "wb") as handle:
        pickle.dump(learner._acq_score_list, handle)



In [11]:

if __name__ == "__main__":
    main()

ww
number of Gold Features 8484
100
200
300
400
500
600
700
800
900
1000
output files ..... 
alignment 0 CDS_aidata


In [18]:
with open("output_dir/CDS_aidata.words1000", "r") as f:
    for line in f:
        print(line.strip())

a:DET,142,14,12,-1,-1,-1,999,
are:AUX,73,11,11,-1,-1,-1,998,
ball:N,41,78,24,-1,-1,-1,932,
be:AUX,64,5,8,5,1,8,1000,
big:ADJ,24,96,26,-1,-1,-1,994,
block:N,54,7,9,-1,-1,-1,998,
blue:ADJ,15,191,27,-1,-1,-1,956,
book:N,51,22,16,-1,-1,-1,986,
can:AUX,72,1,1,-1,-1,-1,997,
car:N,65,13,11,-1,-1,-1,982,
clean:ADJ,24,7,9,7,1,9,980,
close:V,86,6,9,-1,-1,-1,993,
cookie:N,71,4,5,-1,-1,-1,999,
crayon:N,56,18,13,-1,-1,-1,997,
dirty:ADJ,28,26,16,26,1,16,984,
do:AUX,84,21,15,-1,-1,-1,990,
dog:N,60,15,12,-1,-1,-1,992,
draw:V,93,19,14,-1,-1,-1,998,
eat:V,109,5,9,-1,-1,-1,987,
find:V,85,14,12,-1,-1,-1,989,
funny:ADJ,23,41,20,41,1,20,947,
gently:ADV,27,3,4,3,1,4,987,
go:V,82,39,20,-1,-1,-1,985,
happy:ADJ,27,22,15,22,1,15,974,
he:PRON,104,17,12,17,1,12,985,
here:ADV,19,6,9,-1,-1,-1,977,
inside:ADV,22,80,24,80,1,24,996,
is:AUX,68,37,19,37,1,19,994,
it:PRON,107,3,3,3,1,3,1000,
look:V,95,2,3,-1,-1,-1,996,
loud:ADJ,25,2,3,-1,-1,-1,919,
loudly:ADV,21,33,18,33,1,18,958,
my:DET,121,29,17,-1,-1,-1,988,
new:ADJ,30

In [21]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

time_props = pd.read_csv("output_dir/time_props_1000.csv")


display(time_props)




,time,heard,learned,nov_n_min1,nov_n_min2,all,lrn,n,v,oth
0,1,6,3,-1,-1,0.508839,0.999988,0.999988,0.024644,0.507100
1,2,9,3,-1,-1,0.345210,0.999977,0.999954,0.022165,0.343768
2,3,12,5,-1,-1,0.427919,0.999978,0.999954,0.024170,0.507821
3,4,16,7,-1,-1,0.448804,0.999964,0.511371,0.023292,0.606495
4,5,19,9,-1,-1,0.484209,0.999941,0.511371,0.022510,0.672057
5,6,21,9,-1,-1,0.439967,0.999941,0.511371,0.022188,0.621804
6,7,23,10,-1,-1,0.446221,0.999930,0.347941,0.022385,0.648925
7,8,24,11,-1,-1,0.469294,0.999933,0.347941,0.022385,0.672328
8,9,24,11,-1,-1,0.469278,0.999901,0.347941,0.022383,0.672304
9,10,25,11,-1,-1,0.451277,0.999901,0.347941,0.022382,0.631487


In [14]:
with open("output_dir/lex_lm_-1.0_a20.0_ep0.01", "r") as f:
    for line in f:
        print(line.strip())


shoes:N:55 [ ]

<< 0.022330 >>

can:AUX:72 [can#0:(1.00000, 0.00002),  ]

<< 0.000052 >>

what:WH:77 [ ]

<< 0.021668 >>

play:V:86 [cause#0:(0.28669, 0.00001), compete#1:(0.15531, 0.00001), benefit#0:(0.15315, 0.00001), conflict#0:(0.10011, 0.00001), social_interaction#0:(0.09236, 0.00001), exist#0:(0.09189, 0.00001), play:(0.07766, 0.00001), perform#0:(0.04283, 0.00001),  ]

<< 0.000053 >>

she:PRON:115 [ ]

<< 0.018543 >>

your:DET:139 [possessive#0:(0.38422, 0.00001), pronoun#0:(0.35964, 0.04423), 2nd_person#0:(0.17076, 0.00001), your#0:(0.08538, 0.00001),  ]

<< 0.053901 >>

loud:ADJ:25 [attribute#0:(0.56569, 0.00004), physical#0:(0.21242, 0.00004), property#0:(0.18513, 0.23099), sense#0:(0.02827, 0.00004), loud#0:(0.00565, 0.00004), noise#0:(0.00283, 0.00004),  ]

<< 0.189450 >>

look:V:95 [perceive#0:(0.39556, 0.60323), in_reaction_to#0:(0.28746, 0.00001), Pred#0:(0.20533, 0.00001), search#0:(0.08948, 0.00001), look:(0.02217, 0.00001),  ]

<< 0.725516 >>

the:DET:132 [determiner

In [15]:
file_path = "output_dir/aligns_lm_-1.0_a20.0_ep0.01"

with open(file_path, "r") as f:
    for line in f:
        line = line.strip()
        if line and "--" in line:
            print(line)



shoes:N--create#0 [ (19, 0.05042), (168, 0.04980), (284, 0.04146), (297, 0.04397), (411, 0.03286), (514, 0.02812),  ]
shoes:N--perceive#0 [ (2, 0.05040), (277, 0.04757), (497, 0.02628), (540, 0.02814), (562, 0.02669), (586, 0.02484), (665, 0.02268), (879, 0.01812), (907, 0.01808), (976, 0.01710),  ]
shoes:N--cleaning#0 [ (148, 0.04811), (185, 0.05120), (340, 0.04804), (390, 0.04416), (399, 0.04478), (452, 0.04014), (748, 0.02334), (892, 0.02059), (942, 0.02020),  ]
shoes:N--pronoun#0 [ (1, 0.08684), (40, 0.05065), (48, 0.05197), (118, 0.04744), (143, 0.05163), (151, 0.05710), (168, 0.05563), (222, 0.06127), (284, 0.05507), (297, 0.05842), (325, 0.05376), (340, 0.05883), (370, 0.05896), (390, 0.05531), (411, 0.04909), (423, 0.05783), (452, 0.05028), (497, 0.05498), (514, 0.04336), (544, 0.05014), (562, 0.05151), (638, 0.03868), (665, 0.04344), (769, 0.03717), (777, 0.03884), (794, 0.03870), (818, 0.03322), (824, 0.03243), (845, 0.03774), (879, 0.03376), (907, 0.03308), (976, 0.03194),  

In [16]:
import pickle

with open("output_dir/acq_score_timestamp.pkl", "rb") as f:
    acq_scores = pickle.load(f)

for word, time_dict in acq_scores.items():
    print(f"\n Word: {word}")
    for t, score in sorted(time_dict.items()):
        print(f"  Time {t}: {score:.4f}")




 Word: shoes:N
  Time 1: 1.0000
  Time 2: 1.0000
  Time 12: 0.9987
  Time 19: 0.9956
  Time 40: 0.9736
  Time 48: 0.9453
  Time 118: 0.7209
  Time 143: 0.5948
  Time 148: 0.5396
  Time 151: 0.4809
  Time 168: 0.4029
  Time 185: 0.3442
  Time 196: 0.3019
  Time 222: 0.2478
  Time 277: 0.1897
  Time 284: 0.1691
  Time 297: 0.1490
  Time 304: 0.1345
  Time 325: 0.1172
  Time 340: 0.1045
  Time 370: 0.0912
  Time 381: 0.0832
  Time 390: 0.0766
  Time 399: 0.0707
  Time 411: 0.0653
  Time 412: 0.0616
  Time 423: 0.0576
  Time 452: 0.0528
  Time 458: 0.0498
  Time 477: 0.0465
  Time 497: 0.0437
  Time 514: 0.0414
  Time 540: 0.0390
  Time 544: 0.0376
  Time 562: 0.0359
  Time 586: 0.0343
  Time 588: 0.0334
  Time 621: 0.0319
  Time 638: 0.0309
  Time 665: 0.0298
  Time 748: 0.0282
  Time 764: 0.0275
  Time 769: 0.0270
  Time 777: 0.0265
  Time 794: 0.0260
  Time 810: 0.0255
  Time 818: 0.0251
  Time 824: 0.0247
  Time 845: 0.0243
  Time 879: 0.0239
  Time 892: 0.0235
  Time 907: 0.0232
  Ti